# Preprocessing and geocoding 
**! DO NOT RUN** 

In [43]:
import re
import requests
import pandas as pd 

In [44]:
fichier = "constructions_resid"


In [ ]:
#Importation dans le notebook du fichier voulu 
df = pd.read_csv("https://minio.lab.sspcloud.fr/mligeret1/Liste-des-autorisations-durbanisme-creant-des-logements.2025-12.csv", sep=";")

/tmp/ipykernel_239994/2634423762.py:3: DtypeWarning: Columns (0,2,4,6,7,11,16,18,20,21,22,24,30,32,34,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("https://minio.lab.sspcloud.fr/mligeret1/Liste-des-autorisations-durbanisme-creant-des-logements.2025-12.csv", sep=";")


In [3]:
df.columns

Index(['Code de la région du lieu des travaux - Code de la zone',
       'Code de la région du lieu des travaux - Libellé de la zone',
       'Code du département du lieu des travaux - Code de la zone',
       'Code du département du lieu des travaux - Libellé de la zone',
       'Code de la commune du lieu des travaux',
       'Type de DAU -- demande d’autorisation d’urbanisme (PC ou DP généralement, rarement PA)',
       'Numéro d’enregistrement de la DAU', 'Etat d’avancement du projet',
       'Date réelle d’autorisation (PC) ou de non-opposition (DP) initiale',
       'Date réelle d’ouverture de chantier',
       'Date réelle d’achèvement des travaux', 'Année de dépôt de la DAU',
       'Date (mois) de prise en compte (DPC) du premier évènement reçu dans Sitadel (dépôt de la demande ou autorisation)',
       'Date (mois) de prise en compte (DPC) de l’autorisation initiale',
       'Date (mois) de prise en compte (DPC) de l’ouverture du chantier',
       'Date (mois) (DPC) de derniè

Objectif à présent : représenter sur une carte les différents points de construction
  On peut récupérer les géométries des adresses avec cette API
  https://adresse.data.gouv.fr/outils/api-doc/adresse
  https://data.geopf.fr/geocodage/openapi 

In [4]:
elements_adresse = ['Numéro de voie du terrain',
       'Type de voie du terrain', 'Libellé de la voie du terrain',
       'Lieu-dit du terrain', 'Localité du terrain'] 
df_adresses = df.loc[:,elements_adresse]
df['Adresse_complete'] = df_adresses.apply(lambda row : " ".join(row.dropna().astype(str)), axis=1)


In [ ]:
df.head()

,Code de la région du lieu des travaux - Code de la zone,Code de la région du lieu des travaux - Libellé de la zone,Code du département du lieu des travaux - Code de la zone,Code du département du lieu des travaux - Libellé de la zone,Code de la commune du lieu des travaux,"Type de DAU -- demande d'autorisation d'urbanisme (PC ou DP généralement, rarement PA)",Numéro d'enregistrement de la DAU,Etat d'avancement du projet,Date réelle d'autorisation (PC) ou de non-opposition (DP) initiale,Date réelle d'ouverture de chantier,...,Surface de plancher de la destination Entrepôt issue d'une transformation,Surface de plancher de la destination Entrepôt démolie,Surface de plancher de la destination Entrepôt transformée,Surface de plancher de la destination Services publics ou d'intérêt collectif' existante avant travaux,Surface de plancher de la destination Services publics ou d'intérêt collectif nouvelle construite,Surface de plancher de la destination Services publics ou d'intérêt collectif issue d'une transformation,Surface de plancher de la destination Services publics ou d'intérêt collectif démolie,Surface de plancher de la destination Services publics ou d'intérêt collectif transformée,Type de service public ou d'intérêt collectif,Adresse_complete
0,REG_CODE,REG_LIBELLE,DEP_CODE,DEP_LIBELLE,COMM,TYPE_DAU,NUM_DAU,ETAT_DAU,DATE_REELLE_AUTORISATION,DATE_REELLE_DOC,...,SURF_ENT_ISSUE_TRANSFO,SURF_ENT_DEMOLIE,SURF_ENT_TRANSFORMEE,SURF_PUB_AVANT,SURF_PUB_CREEE,SURF_PUB_ISSUE_TRANSFO,SURF_PUB_DEMOLIE,SURF_PUB_TRANSFORMEE,TYPE_SERVICE_PUBLIC,ADR_NUM_TER ADR_TYPEVOIE_TER ADR_LIBVOIE_TER A...
1,82,Rhône-Alpes,01,Ain,01001,PC,00100117V0002,5,2017-05-05,2017-06-01,...,0,0,0,0,0,0,0,0,7,ABERGEMENT CLEMENCIAT
2,82,Rhône-Alpes,01,Ain,01001,PC,00100119V0005,2,2019-08-29,NaN,...,0,0,0,0,0,0,0,0,7,203 Rue des cinq Chemins ABERGEMENT CLEMENCIAT
3,82,Rhône-Alpes,01,Ain,01001,PC,00100120V0003,5,2020-05-25,2024-07-08,...,0,0,0,0,781,0,0,0,6,172 Route du Chateau ABERGEMENT CLEMENCIAT
4,82,Rhône-Alpes,01,Ain,01001,DP,00100119V0011,2,2019-07-22,NaN,...,0,0,0,0,0,0,0,0,7,Sur Brody ABERGEMENT CLEMENCIAT


In [ ]:
df.to_csv(f'{fichier}_preprocessed.csv', encoding='utf-8')




In [ ]:
params = {
    'columns': "Adresse_complete",
}



## Geocding of the whole CSV directly through the API (the entire file but error 413)

In [ ]:

files = {
    'data': open(f'{fichier}_preprocessed.csv', 'rb')
}

response = requests.post(
    'https://api-adresse.data.gouv.fr/search/csv/',
    files=files,
    data=params  
)


In [ ]:
print("Status code :", response.status_code)

OK ? False
Status code : 413


In [16]:
if response.ok:
    with open(f'{fichier}_geocode.csv', 'wb') as f:
        f.write(response.content)

df = pd.read_csv(f'{fichier}_geocode.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'constructions_non_résid_geocode.csv'

## #A COMMENTER Geocoding par chunk 

In [ ]:
dfs_results = [] 
chunksize = 50000
for i, chunk in enumerate(pd.read_csv(f"{fichier}_preprocessed.csv", chunksize=chunksize)):
    print(f"Chunk {i + 1}")

    csv_data = chunk.to_csv(index=False)

    files = {
        "data": ("chunk.csv", csv_data, "text/csv")
    }

    response = requests.post(
        'https://api-adresse.data.gouv.fr/search/csv/',
        files=files,
        data=params,
        timeout=60
    )

    print(response.status_code)

    if not response.ok:
        raise RuntimeError(
            f"error on the chunk {i + 1} "
            f"({response.status_code})"
        )

    # allow us to read the csv returned without saving it each time as a csv 
    df_chunk_geocode = pd.read_csv(
        pd.io.common.BytesIO(response.content)
    )

    dfs_results.append(df_chunk_geocode)
df_final = pd.concat(dfs_results, ignore_index=True)

/tmp/ipykernel_239994/3475115693.py:3: DtypeWarning: Columns (1,3,5,8,12,17,19,21,22,25,31,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(pd.read_csv(f"{fichier}_preprocessed.csv", chunksize=chunksize)):


Chunk 1
200


/tmp/ipykernel_239994/3475115693.py:28: DtypeWarning: Columns (1,3,5,8,12,17,19,21,22,25,31,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,115,118) have mixed types. Specify dtype option on import or set low_memory=False.
  df_chunk_geocode = pd.read_csv(
/tmp/ipykernel_239994/3475115693.py:3: DtypeWarning: Columns (23,31,35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(pd.read_csv(f"{fichier}_preprocessed.csv", chunksize=chunksize)):


Chunk 2
200


/tmp/ipykernel_239994/3475115693.py:28: DtypeWarning: Columns (23,31,35,37,115,118) have mixed types. Specify dtype option on import or set low_memory=False.
  df_chunk_geocode = pd.read_csv(
/tmp/ipykernel_239994/3475115693.py:3: DtypeWarning: Columns (23,31,33,35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(pd.read_csv(f"{fichier}_preprocessed.csv", chunksize=chunksize)):


...

/tmp/ipykernel_239994/3475115693.py:28: DtypeWarning: Columns (23,31,118) have mixed types. Specify dtype option on import or set low_memory=False.
  df_chunk_geocode = pd.read_csv(
/tmp/ipykernel_239994/3475115693.py:3: DtypeWarning: Columns (23,31,33,35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(pd.read_csv(f"{fichier}_preprocessed.csv", chunksize=chunksize)):


Chunk 36
200


/tmp/ipykernel_239994/3475115693.py:28: DtypeWarning: Columns (23,31,33,35,37,109,115,118) have mixed types. Specify dtype option on import or set low_memory=False.
  df_chunk_geocode = pd.read_csv(
/tmp/ipykernel_239994/3475115693.py:3: DtypeWarning: Columns (23,31,33,35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(pd.read_csv(f"{fichier}_preprocessed.csv", chunksize=chunksize)):


Chunk 37
200


/tmp/ipykernel_239994/3475115693.py:28: DtypeWarning: Columns (23,31,33,35,37,115,118) have mixed types. Specify dtype option on import or set low_memory=False.
  df_chunk_geocode = pd.read_csv(


In [ ]:
df_final.to_csv(f"{fichier}_geocoded.csv", index=False, encoding="utf-8")
print(f"creation done")

creation done


## Checking for errors in the geocoding
We first check if there are still some lines that were not geocoded during the batch coding (due to time-out). We send individual request to the API. 

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("https://minio.lab.sspcloud.fr/mligeret1/constructions_resid_geocoded.csv",sep=",",on_bad_lines="warn")


/tmp/ipykernel_459392/3905520969.py:1: DtypeWarning: Columns (1,3,5,7,8,12,17,19,21,22,23,25,31,33,35,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,109,115,118) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("https://minio.lab.sspcloud.fr/mligeret1/constructions_resid_geocoded.csv",sep=",",on_bad_lines="warn")


In [3]:
#we could write a function that would be called "Save", as it saves what's been found in df and separate what's not been found in df_errors, but for lisibility we will keep it as it is 
df_errors = df.loc[~(df["result_status"] == "ok")]
df = df.loc[(df["result_status"] == "ok")]

We look at the number of adresses that were not found

In [4]:
df_errors.loc[~(df_errors["result_status"] == "ok"), "Adresse_complete"]


0          ADR_NUM_TER ADR_TYPEVOIE_TER ADR_LIBVOIE_TER A...
61                      ""LE VILLAGE"" L'ABERGEMENT-DE-VAREY
552        31 LOTIS LE PARC DES DOMBES - LOT 3 AMBERIEUX ...
676                                  ""PRE LEQUIN"" AMBRONAY
703              RUE DES CHENES - LOT B AUX RUETTES AMBRONAY
                                 ...                        
1836639                 L ENTREE DU COLLEGE M ROALE TSINGONI
1836810                               QUART MKAFENI TSINGONI
1836831              RUE DE LA BIBLIOTHEQUE MROALE MAMOUDZOU
1837020                        QUART KITANINI  ZONE TSINGONI
1837102                      18 IMP BALIDJOUMOI COMBANI SADA
Name: Adresse_complete, Length: 48123, dtype: object

We see that by adding too much informations to  ```Adresse complete"```, the API didn't manage to find some adresses. 
To complete the remaining adresse,we will 
- specify the postcode, through ```https://api-adresse.data.gouv.fr/search/?q={street_name}&postcode={postcode}``` (for individual requests), for batch geocoding we just have to specify a postcode column
- look only for the street name and deleting any mention of "Lot" 
- look only for the street name and city name 
- if there are no response only use postcode and take a random street 
(this should not bias the datas as it is probable that only small cities will be taken to this last step, this will be verified)


In [5]:
#We first clea what's happening behind "." as many postcodes are written as "75005.0000"
df_errors["Code postal du terrain"] = (
    df_errors["Code postal du terrain"]
    .astype(str)
    .str.split(".", n=1)
    .str[0]
)


#We then add 0s before the postcode if it has less than  5 digits 
df_errors["Code postal du terrain"] = (df_errors["Code postal du terrain"].str.zfill(5))

#We verify what's left (it's the number of ligns that would have failed to be geocoded by the end of this cleaning part)
df_errors.loc[~df_errors["Code postal du terrain"].astype("str").str.fullmatch(r"\d{5}"),].shape


(786, 120)

As a remark, one can note that most of the adresses beginning with a 0 were not found, as they were coded with 4 digits only. If these lines were deleted, it would have led to bias in the final analysis as constructions datas from departements beginning with a 0 would have been ignored, underestimating construction activity in these departements.

In [6]:
df_errors = df_errors.iloc[:, :df_errors.columns.get_loc("Adresse_complete") + 1]
#We delete the previous columns that were created by the first attempt to geocode 

# TODO / a commenter, 

In [7]:
from io import BytesIO

csv_buffer = BytesIO()
df_errors.to_csv(csv_buffer, index=False)
csv_buffer.seek(0)



0

In [8]:
files = {
    "data": ("df_errors.csv", csv_buffer, "text/csv")
}

params = {
    "columns": "Libellé de la voie du terrain",
    "postcode": "Code postal du terrain",
}



In [9]:
import requests

response = requests.post(
    "https://api-adresse.data.gouv.fr/search/csv/",
    files=files,
    data=params,
    timeout=60
)

response.raise_for_status()


In [10]:
df_errors = pd.read_csv(BytesIO(response.content))


/tmp/ipykernel_459392/2257491172.py:1: DtypeWarning: Columns (1,3,5,8,12,17,19,21,22,25,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,109,115,118) have mixed types. Specify dtype option on import or set low_memory=False.
  df_errors = pd.read_csv(BytesIO(response.content))


In [11]:
df_errors.loc[:,df_errors.columns[100:]]


,Surface de service public ou d’intérêt collectif transformée,Adresse_complete,longitude,latitude,result_score,result_score_next,result_label,result_type,result_id,result_housenumber,result_name,result_street,result_postcode,result_city,result_context,result_citycode,result_oldcitycode,result_oldcity,result_district,result_status
0,SURF_PUB_TRANSFORMEE,ADR_NUM_TER ADR_TYPEVOIE_TER ADR_LIBVOIE_TER A...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,not-found
1,0,"""""LE VILLAGE"""" L'ABERGEMENT-DE-VAREY",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,skipped
2,0,31 LOTIS LE PARC DES DOMBES - LOT 3 AMBERIEUX ...,4.900796,45.993797,0.663503,0.341853,Lotissement le Parc des Dombes 01330 Ambérieux...,street,01005_A067,NaN,Lotissement le Parc des Dombes,Lotissement le Parc des Dombes,1330.0,Ambérieux-en-Dombes,"01, Ain, Auvergne-Rhône-Alpes",1005.0,NaN,NaN,NaN,ok
3,0,"""""PRE LEQUIN"""" AMBRONAY",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,skipped
4,0,RUE DES CHENES - LOT B AUX RUETTES AMBRONAY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,not-found
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48118,0,L ENTREE DU COLLEGE M ROALE TSINGONI,45.130890,-12.789918,0.505805,0.483553,route du collège 97680 Tsingoni,street,97617_0285,NaN,route du collège,route du collège,97680.0,Tsingoni,"976, Mayotte",97617.0,NaN,NaN,NaN,ok
48119,0,QUART MKAFENI TSINGONI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,not-found
48120,0,RUE DE LA BIBLIOTHEQUE MROALE MAMOUDZOU,45.127884,-12.789102,0.785211,NaN,rue de la Bibliothèque 97680 Tsingoni,street,97617_0057,NaN,rue de la Bibliothèque,rue de la Bibliothèque,97680.0,Tsingoni,"976, Mayotte",97617.0,NaN,NaN,NaN,ok
48121,0,QUART KITANINI ZONE TSINGONI,45.135682,-12.788554,0.309020,0.301271,Combani 97680 Tsingoni,street,97617_bb6sxv,NaN,Combani,Combani,97680.0,Tsingoni,"976, Mayotte",97617.0,NaN,NaN,NaN,ok


In [12]:
df_errors.loc[df_errors["longitude"].isna(),].shape

(29406, 120)

There are more than 30 000 points that were not coded yet ...

In [13]:
df = pd.concat([df,df_errors.loc[df_errors["result_status"] == "ok"]],ignore_index=True)

df_errors = df_errors.loc[~(df_errors["result_status"] == "ok")]


In [23]:
df_errors.sample(20).style

,Unnamed: 0,Code de la région du lieu des travaux - Code de la zone,Code de la région du lieu des travaux - Libellé de la zone,Code du département du lieu des travaux - Code de la zone,Code du département du lieu des travaux - Libellé de la zone,Code de la commune du lieu des travaux,"Type de DAU -- demande d’autorisation d’urbanisme (PC ou DP généralement, rarement PA)",Numéro d’enregistrement de la DAU,Etat d’avancement du projet,Date réelle d’autorisation (PC) ou de non-opposition (DP) initiale,Date réelle d’ouverture de chantier,Date réelle d’achèvement des travaux,Année de dépôt de la DAU,Date (mois) de prise en compte (DPC) du premier évènement reçu dans Sitadel (dépôt de la demande ou autorisation),Date (mois) de prise en compte (DPC) de l’autorisation initiale,Date (mois) de prise en compte (DPC) de l’ouverture du chantier,Date (mois) (DPC) de dernière mise à jour des données,Catégorie du demandeur (maître d’ouvrage) selon Sitadel,Code d’activité principale de l’établissement d’un demandeur avéré en tant que personne morale,Catégorie juridique d’un demandeur avéré en tant que personne morale,Dénomination d’un demandeur avéré en tant que personne morale,Numéro SIREN d’un demandeur avéré en tant que personne morale,Numéro SIRET d’un demandeur avéré en tant que personne morale,Code postal du demandeur,Localité du demandeur,Indicateur de recours à un architecte,Numéro de voie du terrain,Type de voie du terrain,Libellé de la voie du terrain,Lieu-dit du terrain,Localité du terrain,Code postal du terrain,Section cadastrale 1,Numéro parcelle cadastrale 1,Section cadastrale 2,Numéro parcelle cadastrale 2,Section cadastrale 3,Numéro parcelle cadastrale 3,Superficie du terrain,Code zone opératoire,Nature du projet déclarée par le demandeur,Nature de projet détaillée,Destination principale,Type principal des logements créés,Type principal de transformation,Type principal des locaux d’origine transformés,Indicateur d’extension,Indicateur de surélévation,Indicateur de création de niveau(x) supplémentaire(s),Nombre de niveaux du bâtiment le plus élevé,Utilisation visée des locaux à construire,Type d’habitation principale ou secondaire,Type d’annexe,Présence d’une piscine dans le projet,Présence d’un garage dans le projet,Présence d’une véranda dans le projet,Présence d’un abri de jardin dans le projet,Présence d’une autre annexe dans le projet,Type de résidence,Présence d’une résidence pour personnes âgées,Présence d’une résidence étudiante,Présence d’une résidence de tourisme,Présence d’une résidence hôtelière à vocation sociale,Présence d’une résidence sociale,Présence d’une résidence pour personnes handicapées,Présence d’un autre type de résidence,Nombre total de logements créés,Nombre de logements individuels créés,Nombre de logements collectifs créés,Nombre de logements individuels purs créés,Nombre de logements individuels groupés créés (hors résidence),Nombre de logements en résidence créés,Nombre de logements collectifs créés hors résidence,Nombre de logements démolis,Nombre de logements de 1 pièce créés,Nombre de logements de 2 pièces créés,Nombre de logements de 3 pièces créés,Nombre de logements de 4 pièces créés,Nombre de logements de 5 pièces créés,Nombre de logements de 6 pièces et plus créés,Nb de logements locatifs sociaux,Nb de logements aidé (hors ptz),Nb de logements ptz,Surface de plancher de la destination'Habitation'existante avant travaux,Surface de plancher de la destination'Habitation'nouvelle construite,Surface de plancher de la destination'Habitation'issue d’une transformation,Surface de plancher de la destination'Habitation'supprimée (démolie),Surface de plancher de la destination'Habitation'transformée,Surface de plancher de locaux non résidentiels (i.e. autres que d’habitation) existante avant travaux,Surface de plancher de locaux non résidentiels (i.e. autres que d’habitation) nouvelle construite,Surface de plancher de locaux non résidentiels (i.e. autres que d’habitation) issue d’une transformation,Sur

In [28]:
df_errors.groupby(
    ["Code de la région du lieu des travaux - Libellé de la zone"]
).size()


Code de la région du lieu des travaux - Libellé de la zone
Alsace                        1046
Aquitaine                     2486
Auvergne                       475
Basse-Normandie               1219
Bourgogne                      365
Bretagne                      2920
Centre                         977
Champagne-Ardenne              449
Corse                         1029
Franche-Comté                  534
Guadeloupe                     121
Guyane                          57
Haute-Normandie                401
La Réunion                     483
Languedoc-Roussillon          4137
Limousin                       181
Lorraine                      1334
Martinique                      85
Mayotte                        176
Midi-Pyrénées                 1666
Nord-Pas-de-Calais            1111
Pays de la Loire              1704
Picardie                       473
Poitou-Charentes              1155
Provence-Alpes-Côte d'Azur    1576
REG_LIBELLE                      1
Rhône-Alpes                   2

We still need to geocode what's left as there are imbalances (places with names that are difficultly written will be underestimated in terms of construction activity)

In [32]:
csv_buffer = BytesIO()
df_errors.to_csv(csv_buffer, index=False)
csv_buffer.seek(0)


0

In [33]:
files = {
    "data": ("df_errors.csv", csv_buffer, "text/csv")
}

params = {
    "columns": "Code postal du terrain",
    "postcode": "Code postal du terrain",
}


In [34]:
response = requests.post(
    "https://api-adresse.data.gouv.fr/search/csv/",
    files=files,
    data=params,
    timeout=60
)

response.raise_for_status()


In [35]:
df_errors = pd.read_csv(BytesIO(response.content))



/tmp/ipykernel_459392/2820757434.py:1: DtypeWarning: Columns (1,3,5,8,12,17,19,21,22,25,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,115) have mixed types. Specify dtype option on import or set low_memory=False.
  df_errors = pd.read_csv(BytesIO(response.content))


In [39]:
df = pd.concat([df,df_errors.loc[df_errors["result_status"] == "ok"]],ignore_index=True)

df_errors = df_errors.loc[~(df_errors["result_status"] == "ok")]

In [ ]:
df_errors.shape

(3084, 120)

In [41]:
df_errors.groupby(
    ["Code de la région du lieu des travaux - Libellé de la zone"]
).size()

Code de la région du lieu des travaux - Libellé de la zone
Alsace                         87
Aquitaine                     130
Auvergne                       47
Basse-Normandie               232
Bourgogne                      18
Bretagne                      155
Centre                         45
Champagne-Ardenne              17
Corse                          31
Franche-Comté                  15
Guadeloupe                     38
Guyane                         15
Haute-Normandie                38
La Réunion                     86
Languedoc-Roussillon          320
Limousin                        5
Lorraine                      180
Martinique                     41
Mayotte                         1
Midi-Pyrénées                  56
Nord-Pas-de-Calais            163
Pays de la Loire              267
Picardie                       12
Poitou-Charentes              110
Provence-Alpes-Côte d'Azur    446
REG_LIBELLE                     1
Rhône-Alpes                   398
Île-de-France          

There are only 3 084 that were not found i.e. 0.1% of the constructions, it can be deemed to be low enough to be deleted without biasing further work. We can check if there are any longitude missing, needed in order to assign hexagons

In [45]:
df.to_csv(f"{fichier}_geocoded_cleaned.csv")

In [46]:
df.loc[df["longitude"].isna()]

,Unnamed: 0,Code de la région du lieu des travaux - Code de la zone,Code de la région du lieu des travaux - Libellé de la zone,Code du département du lieu des travaux - Code de la zone,Code du département du lieu des travaux - Libellé de la zone,Code de la commune du lieu des travaux,"Type de DAU -- demande d’autorisation d’urbanisme (PC ou DP généralement, rarement PA)",Numéro d’enregistrement de la DAU,Etat d’avancement du projet,Date réelle d’autorisation (PC) ou de non-opposition (DP) initiale,...,result_name,result_street,result_postcode,result_city,result_context,result_citycode,result_oldcitycode,result_oldcity,result_district,result_status


## Line-per-line geocoding 

In [ ]:
#Boucle qui permet d'obtenir manuellement les géométries
import requests
def geocodage(adresse) :
        try:
                url = f"https://api-adresse.data.gouv.fr/search/?q={adresse.replace(' ', '+')}"
                data = requests.get(url, timeout=1).json()
        
                if data.get('features'):
                        return data['features'][0]['geometry']['coordinates']
        except:
                pass 


adresses["Géométries"] = adresses["Adresse_complete"].apply(geocodage)

In [ ]:
adresses.head(10)

,Numéro de voie du terrain,Type de voie du terrain,Libellé de la voie du terrain,Lieu-dit du terrain,Localité du terrain,Adresse_complete,Géométries
35693,NaN,NaN,Avenue Jacques Dupin,NaN,PRIVAS,Avenue Jacques Dupin PRIVAS,"[4.611476, 44.722614]"
369554,NaN,Ld,L oiseliere,NaN,La Rouxiere,Ld L oiseliere La Rouxiere,"[-1.089582, 47.44852]"
323031,2,NaN,rue en villeneuve,NaN,Baume-les-Messieurs,2 rue en villeneuve Baume-les-Messieurs,"[5.647772, 46.706858]"
411579,NaN,rue,du Souvenir,NaN,TREMENTINES,rue du Souvenir TREMENTINES,"[-0.787733, 47.119167]"
7462,5,NaN,Route de Meillonnas,TREFFORT,VAL-REVERMONT,5 Route de Meillonnas TREFFORT VAL-REVERMONT,"[5.367165, 46.269018]"
531723,NaN,NaN,MITIGNA OYHANE,NaN,BEYRIE SUR JOYEUSE,MITIGNA OYHANE BEYRIE SUR JOYEUSE,"[-1.07557, 43.303025]"
436936,NaN,NaN,NaN,La Goupillais,La Rouaudiere,La Goupillais La Rouaudiere,"[-1.197518, 47.845472]"
103488,NaN,NaN,le Bourg,NaN,Brie-sous-Chalais,le Bourg Brie-sous-Chalais,"[0.007469, 45.315502]"
335661,622,AV,DES GRANDS LACS,NaN,SANGUINET,622 AV DES GRANDS LACS SANGUINET,"[-1.081388, 44.479231]"
738179,NaN,NaN,chemin d exploitation n 1,L etang des Sanguinots,Courgis,chemin d exploitation n 1 L etang des Sanguin...,"[3.75669, 47.776651]"


#FONCTION POUR TRANSFORMER EN LIGNE UN DICTIONNAIRE 
    json_normalize(
        data=req['features'],  # Quelle liste aplatir
        record_path=['properties'],      # Chemin vers l'objet à "déplier"
        meta=['type'],
        sep='_',
        errors="ignore"                         # Séparateur pour noms de colonnes imbriqués
    )